In [2]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [4]:
from sklearn.svm import LinearSVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

/home/wjkim/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(random_state=42)

In [5]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [8]:
lin_clf.fit(X_train_scaled, y_train)

/home/wjkim/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(random_state=42)

In [10]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9217333333333333

In [12]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [13]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

#### Hyper parameter tuning - Cross validation

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=6.736883323311979, gamma=0.011681728334188778 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. C=6.736883323311979, gamma=0.011681728334188778, total=   0.8s
[CV] C=6.736883323311979, gamma=0.011681728334188778 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .. C=6.736883323311979, gamma=0.011681728334188778, total=   0.9s
[CV] C=6.736883323311979, gamma=0.011681728334188778 .................
[CV] .. C=6.736883323311979, gamma=0.011681728334188778, total=   0.9s
[CV] C=4.372938092810314, gamma=0.023512002458857557 .................
[CV] .. C=4.372938092810314, gamma=0.023512002458857557, total=   0.9s
[CV] C=4.372938092810314, gamma=0.023512002458857557 .................
[CV] .. C=4.372938092810314, gamma=0.023512002458857557, total=   0.9s
[CV] C=4.372938092810314, gamma=0.023512002458857557 .................
[CV] .. C=4.372938092810314, gamma=0.023512002458857557, total=   0.9s
[CV] C=6.678848834087709, gamma=0.028795016720235374 .................
[CV] .. C=6.678848834087709, gamma=0.028795016720235374, total=   0.8s
[CV] C=6.678848834087709, gamma=0.028795016720235374 .................
[CV] .. C=6.678848834087709, gamma=0.028795016720235374, total=   0.9s
[CV] C=6.678848834087709, gamma=0.028795016720235374 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   24.6s finished


RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f59af2f6160>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f59ab901cf8>},
                   verbose=2)

In [17]:
print(rnd_search_cv.best_estimator_)

SVC(C=6.615910315946664, gamma=0.001371552890905965)


In [18]:
print(rnd_search_cv.best_score_)

0.8629917342492193


In [19]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=6.615910315946664, gamma=0.001371552890905965)

In [20]:
y_pred = rnd_search_cvsearch_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9984666666666666

In [21]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9728